In [ ]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install opencv-python ipywidgets imageio

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader
import os
import cv2
import time
from tqdm import tqdm
import wandb

os.environ["TORCH_DEVICE"] = "cuda"
torch.cuda.is_available()

True

# Data Preparation

In [2]:
from pathlib import Path
from datetime import datetime
p = Path('c:/Users/Omar/Documents/ShadowMitigation')
device = os.environ.get("TORCH_DEVICE", 'cuda')

In [3]:

dataset = {}
for path in Path(p, 'SKIPPD').rglob('*.jpg'):
    x = path.stem

    year, month, day, hour, minute = x[:4], x[4:6], x[6:8],x[8:10],x[10:12]
    
    key = f'{year}-{month}-{day}-{hour}'
    if key not in dataset.keys():
        dataset[key] = []
    
    dataset[key].append(path)
    
for key, value in dataset.items():
    dataset[key] = sorted(value)
   

In [4]:
faulty = []
correct = []
for k, v in dataset.items():
    if len(v) != 60:
        
        faulty.append(k)
    else:
        correct.append(k)

In [5]:

SKIPPD = []
for valid in correct:
    print(valid)
    SKIPPD.append([cv2.imread(str(i), cv2.IMREAD_GRAYSCALE) for i in dataset[valid]])


2017-03-09-07
2017-03-09-09
2017-03-09-10
2017-03-09-11
2017-03-09-12
2017-03-09-13
2017-03-09-14
2017-03-09-15
2017-03-09-17
2017-05-12-18
2017-05-13-07
2017-05-13-10
2017-05-13-11
2017-05-13-12
2017-05-13-17
2017-05-14-07
2017-05-14-08
2017-05-14-09
2017-05-14-11
2017-05-14-14
2017-05-14-15
2017-05-14-16
2017-05-14-17
2017-05-14-18
2017-05-15-07
2017-05-15-08
2017-05-15-09
2017-05-15-10
2017-05-15-11
2017-05-15-13
2017-05-15-15
2017-05-15-16
2017-05-15-18
2017-05-16-09
2017-05-16-10
2017-05-16-11
2017-05-16-12
2017-05-16-13
2017-05-16-15
2017-05-16-16
2017-05-16-17
2017-05-16-18
2017-05-18-16
2017-05-18-17
2017-05-18-18
2017-05-19-08
2017-05-19-09
2017-05-19-11
2017-05-19-12
2017-05-19-14
2017-05-19-15
2017-05-19-17
2017-05-19-18
2017-05-20-08
2017-05-20-09
2017-05-20-11
2017-05-20-12
2017-05-20-14
2017-05-20-17
2017-05-21-08
2017-05-21-10
2017-05-21-11
2017-05-21-12
2017-05-21-14
2017-05-21-17
2017-05-21-18
2017-05-22-07
2017-05-22-08
2017-05-22-10
2017-05-22-11
2017-05-22-12
2017-0

In [6]:
from torch.utils.data import DataLoader


SKIPPD_SHORT = SKIPPD[:int(len(SKIPPD))]
# Shuffle Data
np.random.shuffle(SKIPPD_SHORT)


# Train, Test, Validation splits
train_data = SKIPPD_SHORT[:int(0.7*len(SKIPPD_SHORT))]       
val_data = SKIPPD_SHORT[int(0.7*len(SKIPPD_SHORT)):int(0.9*len(SKIPPD_SHORT))]       
test_data = SKIPPD_SHORT[int(0.9*len(SKIPPD_SHORT)):]     

def collate(batch):

    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)     
    batch = batch / 255.0                        
    batch = batch.to(device)                     

    # Randomly pick 15 frames as input, 16th frame is target
    rand = np.random.randint(15,60)                     
    return batch[:,:,rand-15:rand], batch[:,:,rand]     


# # Training Data Loader
train_loader = DataLoader(train_data, shuffle=True, 
                        batch_size=32, collate_fn=collate)

# Validation Data Loader
val_loader = DataLoader(val_data, shuffle=True, 
                        batch_size=32, collate_fn=collate)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

for hour in faulty:
    images =[cv2.imread(str(i), cv2.IMREAD_GRAYSCALE) for i in dataset[hour]]

    for image in images:
        print(image.shape)
    # fig, axs = plt.subplots(8, 8, figsize=(12, 12))

    # for i, ax in enumerate(axs.flat):
    #     if i < len(images):
    #         ax.imshow(images[i], cmap='gray')
    #     ax.axis('off')

    # plt.tight_layout()
    # plt.show()

In [7]:
import io
import imageio
from ipywidgets import widgets, HBox

# Get a batch
input, _ = next(iter(val_loader))

# Reverse process before displaying
input = input.cpu().numpy() * 255.0     

for video in input.squeeze(1)[:3]:          # Loop over videos
    with io.BytesIO() as gif:
        imageio.mimsave(gif,video.astype(np.uint8),"GIF",fps=5)
        display(HBox([widgets.Image(value=gif.getvalue())]))

C:\Users\Omar\AppData\Local\Temp\ipykernel_16820\876004193.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  batch = torch.tensor(batch).unsqueeze(1)


# Model & Training

In [8]:
device = 'cuda'
# The input video frames are grayscale, thus single channel
model = Seq2Seq(num_channels=1, num_kernels=64, 
kernel_size=(3, 3), padding=(1, 1), activation="relu", 
frame_size=(64, 64), num_layers=1).to(device)

optim = Adam(model.parameters(), lr=1e-3)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.BCELoss(reduction='sum')
min_loss = float('inf')

In [9]:
wandb.init(
    # set the wandb project where this run will be logged
    project="ShadowMitigation",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "CONVLSTM",
    "dataset": "SKIPPD",
    "epochs": 200,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: o-abdulmalik (omarmsy). Use `wandb login --relogin` to force relogin


In [10]:
asd = torch.load(r'C:\Users\Omar\Documents\ShadowMitigation\playground\pyt\best.pth')
model.load_state_dict(asd['model_state_dict'])

<All keys matched successfully>

In [20]:
num_epochs = 100
min_loss = 450
optim = Adam(model.parameters(), lr=1e-3)

for epoch in range(1, num_epochs+1):
    start = time.time()
    train_loss = 0                                                 
    model.train()                                                  
    for batch_num, (input_seq, target) in enumerate(tqdm(train_loader), 1):  
        batch_start = time.time()
        output = model(input_seq)          
                                   
        loss = criterion(output.flatten(), target.flatten())       
                           
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()  
                       

        # train_loss += loss.item()


    train_loss /= len(train_loader.dataset)                       
    val_loss = 0                                                 
    model.eval()                                                   
    with torch.no_grad():                                          
        for input, target in val_loader:                          
            output = model(input)                                   
            loss = criterion(output.flatten(), target.flatten())   
            val_loss += loss.item()                                
    val_loss /= len(val_loader.dataset)

    if val_loss < min_loss:
        if val_loss < min_loss or min_loss == 0:                        
            ts = str(time.time()).split(".")[0]
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict(),
                }
            torch.save(checkpoint, f'best_new.pth')
            min_loss = val_loss
            
    wandb.log({"loss": val_loss,"val_loss_perc":val_loss/(64*64) *100})
    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n      Time: {:.2f}".format(
       epoch, train_loss, val_loss, time.time()-start))


100%|██████████| 28/28 [00:20<00:00,  1.40it/s]


Epoch:1 Training Loss:0.00 Validation Loss:1536.28
      Time: 26.67


100%|██████████| 28/28 [00:19<00:00,  1.45it/s]


Epoch:2 Training Loss:0.00 Validation Loss:658.86
      Time: 26.12


100%|██████████| 28/28 [00:20<00:00,  1.40it/s]


...

Epoch:98 Training Loss:0.00 Validation Loss:479.96
      Time: 24.39


100%|██████████| 28/28 [00:18<00:00,  1.54it/s]


Epoch:99 Training Loss:0.00 Validation Loss:477.18
      Time: 24.36


100%|██████████| 28/28 [00:18<00:00,  1.54it/s]


Epoch:100 Training Loss:0.00 Validation Loss:483.62
      Time: 24.43


In [23]:
def collate_test(batch):

    # Last 10 frames are target
    target = np.array(batch)[:,45:]                     
    
    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)          
    batch = batch / 255.0                             
    batch = batch.to(device)                          
    return batch, target

# Test Data Loader

test_loader = DataLoader(test_data,shuffle=True, 
                         batch_size=20, collate_fn=collate_test)

# Get a batch
batch, target = next(iter(test_loader))

# Initialize output sequence
output = np.zeros(target.shape, dtype=np.uint8)

# Loop over timesteps
for timestep in tqdm(range(target.shape[1])):
  input = batch[:,:,timestep:timestep+15]   
  output[:,timestep]=(model(input).squeeze(1).cpu()>0.5)*255.0

100%|██████████| 15/15 [00:02<00:00,  6.59it/s]


In [24]:
from random import shuffle
shuffle(list(zip(target, output)))

for tgt, out in zip(target, output):       # Loop over samples
    
    # Write target video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, tgt, "GIF", fps = 2)    
        target_gif = gif.getvalue()

    # Write output video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, out, "GIF", fps = 2)    
        output_gif = gif.getvalue()

    display(HBox([widgets.Image(value=target_gif), 
                  widgets.Image(value=output_gif)]))